In [3]:
#######################################
#######################################
######### Big Data Programming ########
###### COVID-19 in comparison to ######
### other major diseases since 2000 ###
####### Created by Anthony Asilo ######
############# aka pillared ############
############ December 2020 ############
##### https://github.com/pillared #####
###### https://anthonyasilo.com #######
#######################################
#######################################

from math import *
import statistics
import numpy as np
import pandas as pd
import queue
import time
import csv
import glob
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from geonamescache.mappers import country
from datetime import date


state_abbreviation = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

#Main Method That Provides Data Calculations and Visualizations
def main():
    print("Authenticating...\n")
    time.sleep(1)
    
    #
    #
    # AGGREGATE FILES INTO LIST / DISPLAY EACH FILE NAME AND FILE DATA CONTENTS
    #
    #
    
    directory = "./CLEAN/"                   # directory of all csv files
    files = []                              # list to store all file dataframes

    # Aggregate name and data from csv into a files[] list
    for file_name in glob.glob(directory+'*.csv'):
         df = pd.read_csv(file_name)     # Read file and set to a pandas dataframe
         #print(file_name)                # Print name of file
         files.append((file_name,df))    # Append object containing name of file and dataframe to files[] list

    # Grab contents of element in list and display those contents (name of file and dataframe)       
    for each in files:
        name = each[0]
        data = each[1]
        print(name+"\n")
        print(" | ")
        print(" V ")
        display(data)
        print("----------------------------------------------------------------------------------------------------------------------------\n\n")
    
    #create lists for later... to compare all problems
    #where, name of epidemic, days of epidemic, deaths from epidemic
    where = []
    nameof = []
    daysof = []
    numof = []
    
    #
    #
    # PROBLEM 1a : COVID_USA_CSSE_CLEANED DATA
    #              CHOROPLETH USA DEATHS COVID 19
    #
    
    #Get first file in list of files
    covidusacsse = files[1][1]
    
    # Get Max Cumulative Deaths from each state and add them for total deaths in USA
    d = covidusacsse.groupby("Province_State")["Deaths"].max().sum()
    
    # Convert Date data to date object and sort
    covidusacsse["Date"] = pd.to_datetime(covidusacsse["Date"])
    covidusacsse = covidusacsse.sort_values(by=['Date'])
    
    # Create new DataFrame based on covidusacsse file to plot in choropleth
    df_sample_csse = pd.DataFrame()
    # Get Max cumulative death from each state
    df_sample_csse["TotalDeaths"] = covidusacsse.groupby("Province_State")["Deaths"].max()
    # get each fips code that corresponds to state
    df_sample_csse["FIPS"] = covidusacsse.groupby("Province_State")["FIPS"].unique()
    # Remove int from list
    df_sample_csse["FIPS"] = df_sample_csse['FIPS'].str[0]
    
    print("From %s to %s, there have been a total of %s deaths in the USA from COVID-19...\n" % (covidusacsse["Date"].min(), covidusacsse["Date"].max(), d))  
    print(covidusacsse["Date"].max() - covidusacsse["Date"].min())
    
    # Append valuable items from this data into lists for later consumption and comparison
    where.append("USA")
    nameof.append("USA: COVID-19 (2020)")
    daysof.append((covidusacsse["Date"].max() - covidusacsse["Date"].min()).days)
    numof.append(d)

    # Create new column State and push the row names to the column
    df_sample_csse.index.name = 'State'
    df_sample_csse.reset_index(inplace=True)
    # Append State Code to Column in which Row has State Name for choropleth mapping
    listx = []
    for each in df_sample_csse["State"]:
        listx.append(state_abbreviation[each])
    df_sample_csse["CO"] = listx
    #print(df_sample_csse)
    # Choropleth Data
    fig = go.Figure(data=go.Choropleth(
        locations=df_sample_csse['CO'],
        z=df_sample_csse['TotalDeaths'],
        locationmode="USA-states",
        autocolorscale=False,
        text=df_sample_csse['TotalDeaths'],
        marker_line_color='black',
        colorbar_title="Death Count",
        colorscale=[[0, "rgb(0,68,255)"],
                [0.25, "rgb(0,255,255)"],
                [0.45, "rgb(85,255,0)"],
                [0.65, "rgb(255,239,0)"],
                [0.85, "rgb(255,162,0)"],
                [1, "rgb(255,0,0)"]],
    ))
    fig.update_layout(
        title_text="COVID19 Deaths USA",
        geo = dict(
            scope="usa",
            projection=go.layout.geo.Projection(type='albers usa'),
            showlakes=True,
            lakecolor='rgb(255, 255, 255)'),
    )
    fig.show()
    fig.write_image("./Figures/COVID_USA.png")
    
    #
    #
    # PROBLEM 2a : SARS WORLD DATA
    #
    #
    
    # Get file contents
    df_sars = files[0][1]
    # Convert dates to pythons Date object
    df_sars["Date"] = pd.to_datetime(df_sars["Date"])
    
    print("From %s to %s, there have been a total of %s deaths in the WORLD from SARS...\n" % (df_sars["Date"].min(), df_sars["Date"].max(), df_sars['Number of deaths'].sum()))  
    print(df_sars["Date"].max() - df_sars["Date"].min())
    
    # Append valuable items from this data into lists for later consumption and comparison
    where.append("GLOBAL")
    nameof.append("GLOBAL: SARS (2003)")
    daysof.append((df_sars["Date"].max() - df_sars["Date"].min()).days)
    numof.append(df_sars['Number of deaths'].sum())
    
    # Create Line plot of daily Sars Deaths
    fig_sars = px.line(df_sars, x=df_sars["Date"].unique(), y=df_sars.groupby("Date")["Number of deaths"].sum(), title="Average Profitability per Year")
    fig_sars.update_layout(title = 'Cumulative SARS Deaths (2003 Outbreak)', xaxis_title_text= 'Date',yaxis_title_text= 'Death Count')
    fig_sars.show()
    fig_sars.write_image("./Figures/SARS_WORLD.png")
    
    #
    #
    # PROBLEM 4a / b : HIV WORLD DATA
    #
    #
    
    # Grab File Contents 
    pop = files[2][1]  #Population
    df_hiv = files[3][1]
    
    # Create Alias, noone likes refering to long names!
    df_hiv["Death Rate"] = df_hiv["Deaths - HIV/AIDS - Sex: Both - Age: Age-standardized (Rate) (per 100,000 people)"]
    
    # Remove rows with a null ISO3 Country Code
    df_hiv = df_hiv[~df_hiv['Code'].isnull()] 
    # Remove rows that are not an ISO3 Country Code (3 Letters)
    df_hiv = df_hiv[df_hiv['Code'].map(len) == 3]
    # Remove rows with Taiwans ISO3 as it is not recognized in the Population Data set
    df_hiv = df_hiv[df_hiv['Code'] != "TWN"]
    # Create List for analysis
    deaths_hiv = []
    print("Processing magic...")
    # Basically iterate through both code and year column at same time from HIV data 
    # and cross analyze with population dataset to find actual death number, not rate
    for code, year in zip(df_hiv["Code"], df_hiv["Year"]):
        try:
            deaths_hiv.append( float(df_hiv[(df_hiv["Code"] == str(code)) & (df_hiv["Year"] == int(year))]["Death Rate"]) / 100000 * int(pop[pop["Country Code"] == str(code)][str(year)]) )
        except:
            pass
    # append list created to dataset needed for evaluation
    df_hiv["Deaths"] = deaths_hiv
    
    print("\n\nFrom %s to %s, there have been a total of %s deaths in the WORLD from HIV/AIDS...\n" % (df_hiv["Year"].min(), df_hiv["Year"].max(), df_hiv['Deaths'].sum()))  
    print(365*18,"days")
    
    # Append valuable items from this data into lists for later consumption and comparison
    where.append("GLOBAL")
    nameof.append("GLOBAL: HIV/AIDS (2000+)")
    daysof.append(365*18)
    numof.append(df_hiv['Deaths'].sum())
    
    # Create Series that groups each country and adds death toll
    death_hiv = df_hiv.groupby("Code")["Deaths"].sum()
    # Create Dataframe from Series
    death_hiv = pd.DataFrame(death_hiv)
    # Make Code a column, not an index
    death_hiv.reset_index(inplace=True)
    death_hiv = death_hiv.rename(columns = {'index':'Code'})

    # Use hashmap from country import 
    # and Create list of country Names based on ISO3 Codes
    mapper = country(from_key='iso3', to_key='name')
    _name = []
    for each in death_hiv['Code']:
        name = mapper(each) 
        _name.append(name)
    death_hiv["name"] = _name
    
    # Choropleth of HIV Deaths WOrldwide
    fig_hiv_world = go.Figure(data=go.Choropleth(
        locations = death_hiv['Code'],
        z = death_hiv["Deaths"],
        text = death_hiv['name'],
        autocolorscale=False,
        reversescale=False,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_title = 'Deaths Toll<br>per Country',
        colorscale=[[0, "rgb(0,68,255)"],
                [0.25, "rgb(0,255,255)"],
                [0.45, "rgb(85,255,0)"],
                [0.65, "rgb(255,239,0)"],
                [0.85, "rgb(255,162,0)"],
                [1, "rgb(255,0,0)"]],
    ))

    fig_hiv_world.update_layout(
        title_text='Global HIV Deaths from 2000 - 2017',
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='equirectangular'
        )
    )
    fig_hiv_world.show()
    fig_hiv_world.write_image("./Figures/HIVAIDS_WORLD_MAP.png")
    
    #EXACT DEATHS line graph, not rates
    fig_hiv = px.line(df_hiv, x="Year", y="Deaths", title="", color="Entity")
    fig_hiv.update_layout(title = 'HIV Deaths per country since 2000', xaxis_title_text= 'Date',yaxis_title_text= 'Deaths')
    fig_hiv.show()
    fig_hiv.write_image("./Figures/HIVAIDS_WORLD_BAR.png")
    
    #
    #
    # PROBLEM 5a : EBOLA WORLD DATA
    #
    #
    
    # Grab file contents
    df_ebola = files[4][1]
    # Conver dates to Pythons Date object
    df_ebola["Date"] = pd.to_datetime(df_ebola["Date"])
    # Create Alias, noone wants to refer to long column names!
    df_ebola["cum_deaths"] = df_ebola["Cumulative no. of confirmed, probable and suspected deaths"]         
    
    print("From %s to %s, there have been a total of %s deaths in the WORLD from EBOLA...\n" % (df_ebola["Date"].min(), df_ebola["Date"].max(), df_ebola.groupby("Country")['cum_deaths'].max().sum()))  
    print(df_ebola["Date"].max() - df_ebola["Date"].min())
    
    # Append valuable items from this data into lists for later consumption and comparison
    where.append("GLOBAL")    
    nameof.append("GLOBAL: EBOLA (2014)")
    daysof.append((df_ebola["Date"].max() - df_ebola["Date"].min()).days)
    numof.append(df_ebola.groupby("Country")['cum_deaths'].max().sum())
    
    # Create line graph of ebole deaths worldwide
    fig_ebola = px.line(df_ebola, x="Date", y="cum_deaths", title="", color="Country")
    fig_ebola.update_layout(title = "Cumulative Deaths from Ebola per Country", xaxis_title_text= "Date",yaxis_title_text= "Death Count" )
    fig_ebola.show()
    fig_ebola.write_image("./Figures/EBOLA_WORLD.png")
    
    #
    #
    # PROBLEM 6a, 6b : PNUEMONIA / INFLUENZA USA 
    #
    #
    
    #Retrieve contents from file
    df_pnu_usa = files[5][1]
    
    # Create list seperating Pnuemonia from the rest
    # Summation of each days deaths for cumulative deaths
    pideathsusa = []
    s = 0
    for each in df_pnu_usa["Pneumonia Deaths"]:
        s += each
        pideathsusa.append(s)
    df_pnu_usa["pdeath_cum"] = pideathsusa
    
    # Create list seperating Influenza from the rest
    # Summation of each days deaths for cumulative deaths
    pideathsusa = []
    s = 0
    for each in df_pnu_usa["Influenza Deaths"]:
        s += each
        pideathsusa.append(s)
    df_pnu_usa["ideath_cum"] = pideathsusa
    
    # Basically iterate through both pneumonia and influenza column at same time from data 
    # and reset date for better formattiong with graphs...
    yearweek = []
    for a, b in zip(df_pnu_usa.iloc[:, 0], df_pnu_usa.iloc[:, 1]):
        c = str(a) + "_" + str(b)
        yearweek.append(c)
    df_pnu_usa["yearweek"] = yearweek
    
    print("From %s to %s, there have been a total of %s deaths in the USA from Pneumonia...\n" % (df_pnu_usa["yearweek"].min(), df_pnu_usa["yearweek"].max(), df_pnu_usa['Pneumonia Deaths'].sum()))  
    #Date time does now work for this format... 365*6yrs + 7*21wks = 2337 days
    print((365*6) + (7*21),"days")
    
    # Append valuable items from this data into lists for later consumption and comparison
    where.append("USA")    
    nameof.append("USA: Pneumonia (2000+)")
    daysof.append((365*6) + (7*21))
    numof.append(df_pnu_usa['Pneumonia Deaths'].sum())
    
    # Bar Graph for Pneumina Deaths USA
    fig_pnu_usa = px.bar(df_pnu_usa, x='yearweek', y='pdeath_cum', title='Cumulative Pneumonia Deaths USA')
    fig_pnu_usa.update_layout(title = 'Cumulative Pneumonia Deaths USA', xaxis_title_text= 'Year_Week',yaxis_title_text= 'Death Count')
    fig_pnu_usa.show()
    fig_pnu_usa.write_image("./Figures/PNEUMONIA_USA.png")
    
    print("From %s to %s, there have been a total of %s deaths in the USA from Influenza...\n" % (df_pnu_usa["yearweek"].min(), df_pnu_usa["yearweek"].max(), df_pnu_usa['Influenza Deaths'].sum()))
    #Date time does now work for this format... 365*6yrs + 7*21wks = 2337 days
    print((365*6) + (7*21),"days")
    
    # Append valuable items from this data into lists for later consumption and comparison
    where.append("USA")
    nameof.append("USA: Influenza (2000+)")
    daysof.append((365*6) + (7*21))
    numof.append(df_pnu_usa['Influenza Deaths'].sum())

    # Bar Graph for Influenza USA
    fig_flu_usa = px.bar(df_pnu_usa, x='yearweek', y='ideath_cum', title='Cumulative Influenza Deaths USA')
    fig_flu_usa.update_layout(title = 'Cumulative Influenza Deaths USA', xaxis_title_text= 'Year_Week',yaxis_title_text= 'Death Count')
    fig_flu_usa.show()
    fig_flu_usa.write_image("./Figures/INFLUENZA_USA.png")
    
    #
    #
    # PROBLEM 7a : COVID WORLD 
    #
    #
    
    # Gather File Contents
    df_covid_world = files[6][1]
    
    # Use hashmap from country import 
    # and Create list of ISO3 COdes based on Country Names
    mapper = country(from_key='name', to_key='iso3')
    iso_3 = []
    for each in df_covid_world['Country/Region']:
        iso3 = mapper(each) 
        iso_3.append(iso3)
    df_covid_world["iso3"] = iso_3
    
    # Latest day in dataset has cumulative deaths. grab 
    # and reframe so index is a column with its own name
    df_covid_world_1 = df_covid_world.groupby("Country/Region")["11/25/20"].sum()
    df_covid_world_1 = pd.DataFrame(df_covid_world_1)
    df_covid_world_1.index.name = 'country'
    df_covid_world_1.reset_index(inplace=True)
    
    # Use hashmap from country import 
    # and Create list of ISO3 COdes based on Country Names
    iso_3 = []
    for each in df_covid_world_1['country']:
        iso3 = mapper(each)
        iso_3.append(iso3)
    df_covid_world_1["iso3"] = iso_3
    
    # Get Total Deaths
    deaths_ = df_covid_world["11/25/20"].sum()
    print("From 1/22/20 to 11/25/20, there have been a total of %s deaths in the WORLD from COVID-19...\n" % (deaths_))
    # Formating for days between dates... column names not items, so hardcoded it
    f_date =date(2020,1,22)
    l_date = date(2020,11,25)
    delta = l_date - f_date
    print(delta.days, "days")
    
    # Append valuable items from this data into lists for later consumption and comparison
    where.append("GLOBAL")
    nameof.append("GLOBAL: COVID-19 (2020)")
    daysof.append(delta.days)
    numof.append(deaths_)
    
    # Choropeth COVID WORLD
    fig_covid_world = go.Figure(data=go.Choropleth(
        locations = df_covid_world_1['iso3'],
        z = df_covid_world_1['11/25/20'],
        text = df_covid_world_1['country'],
        autocolorscale=False,
        reversescale=False,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_title = 'Deaths Toll<br>per Country',
        colorscale=[[0, "rgb(0,68,255)"],
                [0.25, "rgb(0,255,255)"],
                [0.45, "rgb(85,255,0)"],
                [0.65, "rgb(255,239,0)"],
                [0.85, "rgb(255,162,0)"],
                [1, "rgb(255,0,0)"]],
    ))

    fig_covid_world.update_layout(
        title_text='Global COVID-19 Deaths from 1/22/2020 - 11/25/20',
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='equirectangular'
        )
    )
    fig_covid_world.show()
    fig_covid_world.write_image("./Figures/COVID_WORLD.png")
        
    #
    #
    # PROBLEM 8a : PNUEMONIA WORLD
    #
    #
    
    # Grab file contents
    df_pne_world = files[7][1]
    
    #Get only columns with deaths (0-4yr, 5-14yr, 15-49yr, 50-69yr, 70+yr)
    ages = df_pne_world.iloc[:,3:8]
    
    # Create list and sum rows columwise, append to original dataset
    pnedeaths = []
    pnedeaths = ages.sum(axis=1)
    df_pne_world["0-70+"] = pnedeaths
    
    # Remove any row that has an empty cell
    df_pne_world = df_pne_world[~df_pne_world['Code'].isnull()] 
    # Remove any row where code is world data and year is less than 2000
    df_pne_world = df_pne_world[(df_pne_world["Code"] != "OWID_WRL") & (df_pne_world["Year"] >= 2000)]

    print("From %s to %s, there have been a total of %s deaths in the WORLD from Pneumonia...\n" % (df_pne_world["Year"].min(), df_pne_world["Year"].max(), df_pne_world['0-70+'].sum()))
    print(365*18,"days")
    
    # Append valuable items from this data into lists for later consumption and comparison
    where.append("GLOBAL")
    nameof.append("GLOBAL: Pnuemonia (2000+)")
    daysof.append(365*18)
    numof.append(df_pne_world['0-70+'].sum())
    
    # Bar Graph Pneumonia WORLD
    fig_pne_world = px.bar(df_pne_world,
                   x=df_pne_world["Year"].unique(), y=df_pne_world.groupby("Year")["0-70+"].sum(), title='Total Deaths Globally by Pneumonia per Year')
    fig_pne_world.update_layout(title = 'Cumulative Deaths from Pnuemonia, Globally', xaxis_title_text= 'Date',yaxis_title_text= 'Death Count')
    fig_pne_world.show()
    fig_pne_world.write_image("./Figures/PNEUMONIA_WORLD.png")

    #
    #
    # PROBLEM 9 DATA AGGREGATION FOR COMPARISONS
    #
    #
    
    # we already concatenated our location, deaths, dates, and names...
    
    dfx = pd.DataFrame()
    dfx["Location"] = where
    dfx["Name"] = nameof
    dfx["Length(Days)"] = daysof
    dfx["TotalDeaths"] = numof
    
    # Create Ratio of Deaths by Length of time passed in outbreak
    dfx["Ratio"] = dfx["TotalDeaths"] / dfx["Length(Days)"]
    display(dfx)

    #
    #
    # PROBLEM 9a-1 USA comparison (Scatter)
    #
    #
    
    dfx_usa = dfx[dfx["Location"] == "USA"]
    fig_dfx_usa = px.scatter(dfx_usa, x = 'Length(Days)', y = 'TotalDeaths', color="Name") 
    fig_dfx_usa.update_layout(title = 'USA Outbreak Comparisons', xaxis_title_text= 'Length (Days)',yaxis_title_text= 'Death Count')    
    fig_dfx_usa.show()
    fig_dfx_usa.write_image("./Figures/COMPARISON-SCATTER_USA.png")
    
    #
    #
    # PROBLEM 9a-2 USA comparison RATIO (Bar)
    #
    #
    
    fig_dfx_usa_2 = px.bar(dfx_usa, x="Name",y="Ratio",title="Ratio by Deaths per Length of Outbreak (USA)")
    fig_dfx_usa_2.show()
    fig_dfx_usa_2.write_image("./Figures/COMPARISON-RATIO_USA.png")
    
    #
    #
    # PROBLEM 9b-1 GLOBAL comparison (Scatter)
    #
    #
    
    dfx_global = dfx[dfx["Location"] == "GLOBAL"]
    fig_dfx_global = px.scatter(dfx_global, x = 'Length(Days)', y = 'TotalDeaths', color="Name") 
    fig_dfx_global.update_layout(title = 'GLOBAL Outbreak Comparisons', xaxis_title_text= 'Length (Days)',yaxis_title_text= 'Death Count')    
    fig_dfx_global.show()
    fig_dfx_global.write_image("./Figures/COMPARISON-SCATTER_WORLD.png")
    
    #
    #
    # PROBLEM 9b-2 GLOBAL comparison RATIO (Bar)
    #
    #
    
    fig_dfx_global_2 = px.bar(dfx_global.sort_values(by='Ratio', ascending=False), x="Name",y="Ratio",title="Ratio by Deaths per Length of Outbreak (GLOBAL)")
    fig_dfx_global_2.show()
    fig_dfx_global_2.write_image("./Figures/COMPARISON-RATIO_WORLD.png")
    
    #
    #
    # PROBLEM 9c-1 BOTH GLOBAL AND USA comparison (Scatter)
    #
    #
    
    fig_dfx = px.scatter(dfx, x = 'Length(Days)', y = 'TotalDeaths', color="Name") 
    fig_dfx.update_layout(title = 'BOTH GLOBAL AND USA Outbreak Comparisons', xaxis_title_text= 'Length (Days)',yaxis_title_text= 'Death Count')    
    fig_dfx.show()
    fig_dfx.write_image("./Figures/COMPARISON-SCATTER_BOTH.png")
    
    #
    #
    # PROBLEM 9c-2 BOTH GLOBAL AND USA comparison RATIO (Bar)
    #
    #
    
    fig_dfx_2 = px.bar(dfx.sort_values(by='Ratio', ascending=False), x="Name",y="Ratio",title="Ratio by Deaths per Length of Outbreak (BOTH GLOBAL AND USA)")
    fig_dfx_2.show()
    fig_dfx_2.write_image("./Figures/COMPARISON-RATIO_BOTH.png")
     
#Caller    
if __name__ == "__main__":
    main()   

Authenticating...

./CLEAN/SARS_WORLD.csv

 | 
 V 


,Date,Country,Cumulative number of case(s),Number of deaths,Number recovered
0,2003-03-17,Germany,1,0,0
1,2003-03-17,Canada,8,2,0
2,2003-03-17,Singapore,20,0,0
3,2003-03-17,"Hong Kong SAR, China",95,1,0
4,2003-03-17,Switzerland,2,0,0
...,...,...,...,...,...
2533,2003-07-11,Switzerland,1,0,1
2534,2003-07-11,Thailand,9,2,7
2535,2003-07-11,United Kingdom,4,0,4
2536,2003-07-11,United States,75,0,67


----------------------------------------------------------------------------------------------------------------------------


./CLEAN/COVID_USA.csv

 | 
 V 


,i,Province_State,Confirmed,Deaths,FIPS,Mortality_Rate,Date
0,0,Alabama,20777,692,1,3.375610,6/7/20
1,1,Alaska,543,10,2,1.841621,6/7/20
2,2,Arizona,26989,1051,4,3.894179,6/7/20
3,3,Arkansas,9426,154,5,1.633779,6/7/20
4,4,California,130615,4632,6,3.546300,6/7/20
...,...,...,...,...,...,...,...
11623,11623,Virginia,11594,411,51,3.544937,4/24/20
11624,11624,Washington,12977,722,53,5.563690,4/24/20
11625,11625,West Virginia,1010,32,54,3.168317,4/24/20
11626,11626,Wisconsin,5356,263,55,4.910381,4/24/20


----------------------------------------------------------------------------------------------------------------------------


./CLEAN/WORLD_POPULATIONS.csv

 | 
 V 


...

From 2013_40 to 2020_9, there have been a total of 1331647 deaths in the USA from Pneumonia...

2337 days


From 2013_40 to 2020_9, there have been a total of 55259 deaths in the USA from Influenza...

2337 days


From 1/22/20 to 11/25/20, there have been a total of 1421308 deaths in the WORLD from COVID-19...

308 days


From 2000 to 2017, there have been a total of 48378423.41746253 deaths in the WORLD from Pneumonia...

6570 days


,Location,Name,Length(Days),TotalDeaths,Ratio
0,USA,USA: COVID-19 (2020),227,2.610330e+05,1149.925110
1,GLOBAL,GLOBAL: SARS (2003),116,4.520900e+04,389.732759
2,GLOBAL,GLOBAL: HIV/AIDS (2000+),6570,3.343616e+07,5089.218280
3,GLOBAL,GLOBAL: EBOLA (2014),572,1.131200e+04,19.776224
4,USA,USA: Pneumonia (2000+),2337,1.331647e+06,569.810441
5,USA,USA: Influenza (2000+),2337,5.525900e+04,23.645272
6,GLOBAL,GLOBAL: COVID-19 (2020),308,1.421308e+06,4614.636364
7,GLOBAL,GLOBAL: Pnuemonia (2000+),6570,4.837842e+07,7363.534767
